<a href="https://colab.research.google.com/github/CristinaMarsh/Reproduce/blob/main/Uncertaintyforecasting/2023_3_13_wind_power_forecasting_SVGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Thanks https://github.com/honglinwen/Wind-power-forecasting-based-on-SVGP/blob/main/wind_power_forecasting_SVGP.ipynb

In [1]:
! git clone https://github.com/honglinwen/Wind-power-forecasting-based-on-SVGP.git

Cloning into 'Wind-power-forecasting-based-on-SVGP'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 747.20 KiB | 2.24 MiB/s, done.


In [2]:
%cd Wind-power-forecasting-based-on-SVGP

/content/Wind-power-forecasting-based-on-SVGP


In [3]:
!pip install gpytorch
!pip install tqdm
!pip install -U scikit-learn
!pip install properscoring

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 71.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:

import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tqdm
import sklearn
import copy
import properscoring as ps
%matplotlib inline
%load_ext autoreload
%autoreload 2
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.distributions.transforms import SigmoidTransform, PowerTransform
from torch.distributions.normal import Normal
     

In [6]:

def fill_ndarray(t1):
    for i in range(t1.shape[1]):  
        temp_col = t1[:, i]  
        nan_num = np.count_nonzero(temp_col != temp_col)
        if nan_num != 0:  
            temp_not_nan_col = temp_col[temp_col == temp_col]  
            temp_col[np.isnan(temp_col)] = temp_not_nan_col.mean()  
    return t1

In [7]:
t1 = np.array([[1, 2, np.nan], [4, np.nan, 6], [7, 8, 9]])
print("Original array:")
print(t1)

t2 = fill_ndarray(t1)
print("Array after filling NaN values:")
print(t2)

Original array:
[[ 1.  2. nan]
 [ 4. nan  6.]
 [ 7.  8.  9.]]
Array after filling NaN values:
[[1.  2.  7.5]
 [4.  5.  6. ]
 [7.  8.  9. ]]


In [8]:
def get_data(df):
  data = df['TARGETVAR'].values.reshape((-1,1))
  return data

def get_nwp(d):
  cls = d.columns
  data = []
  for i in range(4):
    data.append(d[cls[i+3]].values.reshape((-1,1)))
  data = np.hstack(data)
  return data

data = pd.read_csv('/content/Wind-power-forecasting-based-on-SVGP/Task15_W_Zone1.csv',delimiter=',')

power = get_data(data); x = get_nwp(data)
y = fill_ndarray(power).flatten()
y_true = copy.deepcopy(y)


In [9]:

v = 1
for i in range(len(y)):
    if y[i] == 0:
        y[i] = 1e-3
        y_true[i] = 1e-3
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
    elif y[i] == 1:
        y[i] = 1-1e-3
        y_true[i] = 1-1e-3
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
    else:
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
y = torch.tensor(y).float()
x = torch.tensor(x).float()

In [10]:
samples = math.floor(x.shape[0]*0.8)

train_y = y[:samples]
train_x = x[:samples,:]


y_true = y_true[samples:]
test_y = y[samples:]
test_x = x[samples:,:]

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
     


In [11]:
math.floor?

In [13]:
samples

13440

In [14]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

In [15]:
from torch import nn
class MLPMean(gpytorch.means.Mean):
    def __init__(self, input_size):
        super(MLPMean, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),  
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(), 
            nn.Linear(512, 1))

        count = 0
        for n, p in self.mlp.named_parameters():
            self.register_parameter(name = 'mlp' + str(count), parameter = p)
            count += 1
    
    def forward(self, x):
        m = self.mlp(x)
        return m.squeeze()
     

In [16]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)
        
        self.mean_module = MLPMean(input_size=4)
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=4))
        

    def forward(self, x):
        x = self.scale_to_bounds(x)
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = train_x[torch.randperm(train_x.size(0))[:1000]]
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()
     

In [17]:
num_epochs = 200
losses = []

model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[50,100], gamma=0.5)

mll = gpytorch.mlls.PredictiveLogLikelihood(likelihood, model, num_data=train_y.size(0))


epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    minibatch_iter = tqdm.notebook.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        optimizer.zero_grad()
        output = likelihood(model(x_batch))
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()
    scheduler.step()
    losses.append(loss.item())

     

Epoch:   0%|          | 0/200 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

...

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/14 [00:00<?, ?it/s]

这段代码是使用GPyTorch库实现的训练循环，该库是基于PyTorch的高斯过程（Gaussian Process）建模工具包。

第一行代码定义了一个PredictiveLogLikelihood类的对象mll，该对象用于计算预测值的对数似然。该对象需要三个参数，分别为likelihood（代表高斯过程的似然函数）、model（代表高斯过程的模型）和num_data（代表训练数据的个数）。

接下来的代码使用两个for循环进行训练，其中外层循环控制训练的轮数，内层循环则控制每个批次数据的训练。具体来说，内层循环遍历训练数据集中的所有批次，每次循环从训练数据加载器train_loader中取出一个批次的数据x_batch和y_batch，分别表示输入数据和对应的目标值。然后将优化器optimizer的梯度清零（因为PyTorch默认会累积梯度），然后使用模型model对x_batch进行预测，得到一个输出output。接下来，使用前面定义的mll对象对输出output和对应的目标值y_batch计算对数似然，并将似然取反作为损失函数loss。最后，调用loss的backward方法计算损失函数的梯度，并使用优化器optimizer更新模型的参数。在每个批次训练结束后，使用tqdm库中的set_postfix方法更新内层循环进度条的显示信息，显示损失函数的值。

最后，在外层循环结束后，调用scheduler.step()方法更新优化器的学习率，并将当前轮的损失函数值loss.item()添加到losses列表中。

总之，该代码是一个使用GPyTorch实现的高斯过程模型的训练循环，其中使用了PredictiveLogLikelihood类计算对数似然，并使用PyTorch中的优化器和学习率调度器对模型进行优化。

In [18]:

model.eval()
likelihood.eval()

mean = torch.tensor([0.]); std = torch.tensor([0.]); f_std = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = likelihood(model(x_batch))
        mean = torch.cat([mean, preds.mean.cpu()])
        std = torch.cat([std, preds.stddev.cpu()])
        f_std = torch.cat([f_std, model(x_batch).stddev.cpu()])
mean = mean[1:]; std = std[1:]; f_std = f_std[1:]
      
      

In [19]:
mean = mean.cpu()
std = std.cpu()
f_std = f_std.cpu()

train_x = train_x.cpu()
train_y = train_y.cpu()
test_x = test_x.cpu()
test_y = test_y.cpu()
     

In [20]:
length = len(y_true)
score = 0 
for i in range(length):
  m = torch.distributions.normal.Normal(mean[i],std[i])
  transforms = [SigmoidTransform(),PowerTransform(1)]
  logit_normal = TransformedDistribution(m, transforms)
  sample = logit_normal.sample([1000]).numpy()
  score += ps.crps_ensemble(y_true[i] , sample)

score = score/length
print(score)

0.09226549537348674
